In [2]:
import json
import os
import re
from langdetect import detect
from langdetect import detect_langs
from tqdm import tqdm
pattern_list_zh=['（?另见[^）]*.',
              r'看?法? 进行患者培训 ',
              '[^\。]*参考(文献)*.?doi: [^\s]+',
              '[^\。]*参考(文献)?[^，。]*.',
              '[^,。~]*... 阅读更多[^\u4e00-\u9fa5]*',
              '更多信息[^，。,\.]*.',
              'www\.\w+\.\w+[^\s]*\s',
              '会在这本.*点击此处',
              '（?(也可)?参见(概述)?[^\.,，）]*.）?',
              '[^\.,]*\.\.\. Common.TooltipReadMore',
              '(表格)?(Table)? \$\(document\).*\);',
              '请注意，本手册.*',
              '\(又见 .*[^ ]概述{1,3}',
              '\(s?S?ee[^\)]*.\.?',
              '观看这些视频.*',
              #r'(\p{L}+)\s+\1(\s+\1)*',
              ' ?\(?（?见表 ?[^。]*',
              ' ?（?\(?见图 ?[^。]*',
              ' ?（?\(?也见 ?[^。]*',
              '[\(（ 也可同样]+见 ?[^。]*。?',
              '[^\u4e00-\u9fa5-。]{10,1000}',
            ]
pattern_list_en=['[^\.]*R?r?eferences? [^，。]*.?doi:?\.? ?[^\s]+',
                 #r'\b(\w+)\s+\1\b(\s+\1)*',
                 '[^.]*\(S?s?ee[^\.]*\)?(...)?( Common.TooltipReadMore)?[^\.]*\.',
                 ' ?View Patient Education ',
                 '(\(\d\s+References)?(\(See)?.*\.\.\.\sread more\s+\)\.',
                 'More Information The following.*',
                 '(表格)?(Table)? \$\(document\).*\);',
                 '[^.]*\(S?s?ee[^\.]*\)?',
                 '[^\.]*([\.]{1,10})? ?read more[^\.\?\!\s]*',
                 '（?另见[^）]*.',
                 '[^\.]*([\.]{1,10})? ?Common.TooltipReadMore[^\w^\(]*',' ?（?\(?见图 ?[^。]*']

pattern_ngram_en = r'([A-Za-z]+)\s+\1(\s+\1)*'
pattern_ngram_zh = r'\b(\w+)\s+\1\b(\s+\1)*'
pattern_ngram_punc = r'([\.。]+\s?)\1*'

In [3]:
def detect_language(content):
    #print("context",content)
    lang = detect(content)
    if lang == "zh-cn":
        return "zh"
    if lang == "en":
        return "en"
    return "None"

no


In [10]:
def mv_ngram_repeat(content,pattern_ngram,lang):
    part = re.findall(pattern_ngram, content, flags=re.IGNORECASE)
    if not part:
        return content
    if lang == "zh":
        for p in part:
            content = re.sub(rf'({p[0]})\s+\1(\s+\1)*', p[0], content,flags=re.IGNORECASE)
            # content=re.sub('通常需要精神病转诊。|AAABBBCCC|<><><>|ABCBCABAC|又见 脊髓病变疾病概述.）...|','',content)
    if lang == "en":
        for p in part:
            content = re.sub(rf'\b({p[0]})\s+\1\b(\s+\1)*', p[0], content,flags=re.IGNORECASE)
            # item=re.sub('© Springer Science+Business Media|Did You Know...','',item)
    return content            

def process_cite(context,pattern_list,pattern_ngram,lang):
    result = []
    split_token = "\n\n"
    if split_token not in context:
        split_token = "\n"
    for item in context.split(split_token):
        item = item.strip(split_token)
        item = mv_ngram_repeat(item,pattern_ngram,lang)
        item = re.sub('通常需要精神病转诊。|AAABBBCCC|<><><>|ABCBCABAC|又见 脊髓病变疾病概述.）...|','',item)
        item = re.sub('© Springer Science+Business Media|Did You Know...','',item)
        for pattern_item in pattern_list:
            item = re.sub(pattern_item, '', item, flags=re.IGNORECASE)
            #item = re.findall('([^-.\dwIX]{1,10})\1{2,}', item, flags=re.IGNORECASE)
            #print(pattern_item, "\t",item)
        result.append(item)
    return split_token.join(result)

def rm_last_paragh(text):
    context = text.split("。")
    if len(context[-1]) < 5:
        context = context[:-1]
        context[-1] = context[-1] + "。"
    new_context = []
    for item in context:
        if len(item) > 4:
            new_context.append(item)
    return "。".join(new_context)

def rm_special_char(text):
    if text[-2:] == "了解":
        return text[:-2]
    return text

In [47]:
fw = open("msd_full_clean.jsonl","w",encoding="utf-8")
with open("msd_full.jsonl","r",encoding="utf-8") as fs:
    for items in tqdm(fs.readlines()):
        item = json.loads(items.strip())
        context = item["text"]
        # if item["seq_id"] != 9165:
        #     continue
        #print(item)
        lang = detect_language(item["text"])
        # print(context)
        # print("========================================")
        if lang == "zh":
            clean_text = process_cite(context,pattern_list_zh,pattern_ngram_zh,lang)
            clean_text = re.sub(pattern_ngram_punc, "。", clean_text)
            clean_text = clean_text.replace("。，","。")
            clean_text = rm_last_paragh(clean_text)
        else:
            clean_text = process_cite(context,pattern_list_en,pattern_ngram_en,lang)
            clean_text = re.sub(pattern_ngram_punc, ".", clean_text)
            # 替换标点连用
            clean_text = re.sub(r"\.\s?,\s?",".", clean_text)
        #print(clean_text)
        clean_text = rm_special_char(clean_text)
        item["text"] = clean_text
        item["lang"] = lang
        item["tokens"] = ""
        item = json.dumps(item,ensure_ascii=False)
        fw.write(item+"\n")
        #break

100%|██████████| 8149/8149 [08:24<00:00, 16.16it/s]
